# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [2]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [3]:
# frequency table for prestige and whether or not someone was admitted
df['counter']= 1
pd.pivot_table(df,values='counter', index=['prestige'],columns=['admit'],aggfunc=np.sum)

C:\Users\Marc\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


admit,0,1
prestige,,
1.0,28,33
2.0,95,53
3.0,93,28
4.0,55,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [4]:
df = df.drop('counter',axis=1)
df = pd.get_dummies(df,columns=['prestige'],drop_first=False)
#df = df.drop('prestige_4.0',axis=1)
df.head()

,admit,gre,gpa,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0,0,1,0
1,1,660.0,3.67,0,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,0,0,0,1
4,0,520.0,2.93,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: We need 5 class variables. They are GRE, GPA, prestige_2, prestige_3, and prestige_4. With the prestige variable, we dropped the first category as a dummy variable and therefore it shouldn't appear in the model.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [5]:
#crosstab prestige 1 admission 
#frequency table cutting prestige and whether or not someone was admitted
handCalc = pd.crosstab(df['prestige_1.0'],df['admit'],)
handCalc

admit,0,1
prestige_1.0,,
0,243,93
1,28,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [6]:
prob1 = handCalc.ix[1,1]/float(handCalc.ix[1,0]+handCalc.ix[1,1])
odds1 = prob1/(1-prob1)
print odds1

1.17857142857


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [7]:
prob2 = handCalc.ix[0,1]/float(handCalc.ix[0,0]+handCalc.ix[0,1])
odds2 = prob2/(1-prob2)
print odds2

0.382716049383


#### 3.3 Calculate the odds ratio

In [8]:
oddsRatio = odds1/float(odds2)
print oddsRatio

3.07949308756


#### 3.4 Write this finding in a sentence: 

Answer: An individual's odds of being admitted to grad school vs. not being admitted given that they've attended a #1 ranked college are roughly 3.08:1. In other words, an individual who has gone to a #1 ranked school is more likely to be admitted to grad school than to not be admitted.

#### 3.5 Print the cross tab for prestige_4

In [9]:
#crosstab prestige 1 admission 
#frequency table cutting prestige and whether or not someone was admitted
handCalc2 = pd.crosstab(df['prestige_4.0'],df['admit'],)
handCalc2

admit,0,1
prestige_4.0,,
0,216,114
1,55,12


#### 3.6 Calculate the OR 

In [10]:
prob3 = handCalc2.ix[1,1]/float(handCalc2.ix[1,0]+handCalc2.ix[1,1])
odds3 = prob3/(1-prob3)
#print odds3   # prob of being admitted to grad school given attended a rank 4 school

prob4 = handCalc2.ix[1,0]/float(handCalc2.ix[1,0]+handCalc2.ix[1,1])
odds4 = prob4/(1-prob4)
#print odds4   # prob of not being admitted to grad school given attended a rank 4 school

oddsRatio2 = odds3/float(odds4)
print oddsRatio2

0.0476033057851


#### 3.7 Write this finding in a sentence

Answer: An individual's odds of being admitted to grad school vs. not being admitted given that they've attended a #4 ranked college are roughly .048:1. In other words, an individual who has gone to a #4 ranked school is less likely to be admitted to grad school than to be admitted.

## Part 4. Analysis

In [12]:
# create a clean data frame for the regression
data = df.drop('prestige_1.0',axis=1)
data.head()

,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0,1,0
1,1,660.0,3.67,0,1,0
2,1,800.0,4.00,0,0,0
3,1,640.0,3.19,0,0,1
4,0,520.0,2.93,0,0,1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [13]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [35]:
train_cols = ['gre', 'gpa', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0', 'intercept']

#### 4.2 Fit the model

In [36]:
logit = sm.Logit(data['admit'], data[train_cols])

# fit the model
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [37]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Mon, 22 May 2017   Pseudo R-squ.:                 0.08166
Time:                        20:11:22   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2.0    -0.6801      0.317     -2.14

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [39]:
print np.exp(result.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64


In [41]:
# odds ratios and 95% CI
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

                  2.5%     97.5%        OR
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_2.0  0.272168  0.942767  0.506548
prestige_3.0  0.133377  0.515419  0.262192
prestige_4.0  0.093329  0.479411  0.211525
intercept     0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: The odds ratio of Prestige_2 is ~.507, meaning that attending a second-ranked school is associated with lower odds of being admitted to grad school.

#### 4.6 Interpret the OR of GPA

Answer: The odds ratio of GPA is ~2.18, meaning that improving GPA is associated with higher odds of being admitted to grad school.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [78]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [79]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres

gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas

# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [80]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
combos = pd.get_dummies(combos, columns=['prestige'], prefix='prestige',drop_first=True)

#### 5.2 Make predictions on the enumerated dataset

In [81]:
# make predictions on the enumerated dataset
combos['admit_pred'] = result.predict(combos)

print combos.tail(4)

       gre  gpa  intercept  prestige_2.0  prestige_3.0  prestige_4.0  \
396  800.0  4.0        1.0             0             0             0   
397  800.0  4.0        1.0             1             0             0   
398  800.0  4.0        1.0             0             1             0   
399  800.0  4.0        1.0             0             0             1   

     admit_pred  
396    0.985399  
397    0.946509  
398    0.934534  
399    0.582995  


#### 5.3 Interpret findings for the last 4 observations

Answer: From the last 4 observations, we can deduce that attending higher-ranked schools is positively associated with graduate school admittance. We see this from the fact that the predictions get further from 1 as prestige goes down, ceteris paribus.

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.